In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from sklearn.utils.multiclass import unique_labels

from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.utils import resample


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
#### read in the flattened dataframes with the labels for the 98 to 2015 data
MSFT_df_98_to_15_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/MSFT_flat_98_to_2015.csv")
AAPL_df_98_to_15_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/AAPL_flat_98_to_2015.csv")
AMZN_df_98_to_15_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/AMZN_flat_98_to_2015.csv")

In [3]:
###read in the flattened dataframes with the labels for 2015 to 2019
MSFT_df_15_to_19_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/MSFT_flat_2015_2019.csv")
AAPL_df_15_to_19_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/AAPL_flat_2015_2019.csv")
AMZN_df_15_to_19_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/AMZN_flat_2015_2019.csv")



In [4]:
#vertically stack the MSFT and AAPL dataframes on top of eachother
df = pd.concat([MSFT_df_98_to_15_df, AAPL_df_98_to_15_df], axis=0)
#vertically stack the MSFT&AAPL df and the AMZN dataframe ontop of eachother
df = pd.concat([df,AMZN_df_98_to_15_df],axis = 0)

df = df.drop(['Unnamed: 0'], axis = 1)

X_train = df.drop(['Label'], axis =1)


In [5]:
y_train = pd.read_csv(r"y_train_multiclass_labels.csv")
y_train = y_train.drop(['Unnamed: 0'], axis=1)

In [6]:
df_2 = pd.concat([MSFT_df_15_to_19_df, AAPL_df_15_to_19_df], axis=0)
df_2 = pd.concat([df_2,AMZN_df_15_to_19_df],axis = 0)
dates = df_2['Unnamed: 0']

df_2 = df_2.drop(['Unnamed: 0'], axis = 1)

#get labels for the prediction data
y_test = df_2['Label']

#drop the labelss
df_2 = df_2.drop(['Label'], axis=1)

#X_test to predict the values
X_test = df_2

In [7]:
from keras.utils import to_categorical
encoded = to_categorical(y_train)
y_train = encoded

In [8]:
def make_timeseries_classifier(sample_num,window_size, filter_length, nb_input_series=1, nb_outputs=4, nb_filter=64):
    """:Return: a Keras Model for predicting the next value in a timeseries given a fixed-size lookback window of previous values.
    The model can handle multiple input timeseries (`nb_input_series`) and multiple prediction targets (`nb_outputs`).
    :param int window_size: The number of previous timeseries values to use as input features.  Also called lag or lookback.
    :param int nb_input_series: The number of input timeseries; 1 for a single timeseries.
      The `X` input to ``fit()`` should be an array of shape ``(n_instances, window_size, nb_input_series)``; each instance is
      a 2D array of shape ``(window_size, nb_input_series)``.  For example, for `window_size` = 3 and `nb_input_series` = 1 (a
      single timeseries), one instance could be ``[[0], [1], [2]]``. See ``make_timeseries_instances()``.
    :param int nb_outputs: The output dimension, often equal to the number of inputs.
      For each input instance (array with shape ``(window_size, nb_input_series)``), the output is a vector of size `nb_outputs`,
      usually the value(s) predicted to come after the last value in that input instance, i.e., the next value
      in the sequence. The `y` input to ``fit()`` should be an array of shape ``(n_instances, nb_outputs)``.
    :param int filter_length: the size (along the `window_size` dimension) of the sliding window that gets convolved with
      each position along each instance. The difference between 1D and 2D convolution is that a 1D filter's "height" is fixed
      to the number of input timeseries (its "width" being `filter_length`), and it can only slide along the window
      dimension.  This is useful as generally the input timeseries have no spatial/ordinal relationship, so it's not
      meaningful to look for patterns that are invariant with respect to subsets of the timeseries.
    :param int nb_filter: The number of different filters to learn (roughly, input patterns to recognize).
    """
    n_timesteps, n_features, n_outputs =1,360, 4
    
    model = Sequential()
    model.add(Convolution1D(filters=64, kernel_size=3, activation='relu',
    input_shape=(window_size,nb_input_series)))
    model.add(Convolution1D(filters=64, kernel_size=3, activation='relu')) 
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(4, activation='softmax')) 
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    # To perform (binary) classification instead:
    # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    
    return model

In [9]:
model_1 = make_timeseries_classifier(X_train.shape[0],X_train.shape[1],3)

W0810 19:07:26.185464 4633925056 deprecation_wrapper.py:119] From /Users/fahad/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0810 19:07:26.203205 4633925056 deprecation_wrapper.py:119] From /Users/fahad/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0810 19:07:26.206294 4633925056 deprecation_wrapper.py:119] From /Users/fahad/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 19:07:26.244357 4633925056 deprecation_wrapper.py:119] From /Users/fahad/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default inst

In [10]:
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 358, 64)           256       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 356, 64)           12352     
_________________________________________________________________
dropout_1 (Dropout)          (None, 356, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 178, 64)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 11392)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1139300   
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total para

In [11]:
es = EarlyStopping(monitor='val_acc', mode='max', verbose=0, patience=150)
mc = ModelCheckpoint('1dconv_feature_engineered_model_multiclass.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
cb_list = [es,mc]

In [12]:
history=model_1.fit(np.array(X_train).reshape((*X_train.shape,1)),
                    y_train,
            validation_split=0.10, epochs=1000
            ,batch_size=128, verbose=1, callbacks = cb_list)

W0810 19:07:26.425268 4633925056 deprecation.py:323] From /Users/fahad/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 550 samples, validate on 62 samples
Epoch 1/1000
550/550 [==============================] - 1s 2ms/step - loss: 1.3798 - acc: 0.3109 - val_loss: 1.2160 - val_acc: 0.5161

Epoch 00001: val_acc improved from -inf to 0.51613, saving model to 1dconv_feature_engineered_model_multiclass.h5
Epoch 2/1000
550/550 [==============================] - 0s 812us/step - loss: 1.3542 - acc: 0.3564 - val_loss: 1.3623 - val_acc: 0.3710

Epoch 00002: val_acc did not improve from 0.51613
Epoch 3/1000
550/550 [==============================] - 0s 797us/step - loss: 1.3340 - acc: 0.3709 - val_loss: 1.4400 - val_acc: 0.2097

Epoch 00003: val_acc did not improve from 0.51613
Epoch 4/1000
550/550 [==============================] - 0s 762us/step - loss: 1.3083 - acc: 0.4109 - val_loss: 1.4729 - val_acc: 0.2581

Epoch 00004: val_acc did not improve from 0.51613
Epoch 5/1000
550/550 [==============================] - 0s 803us/step - loss: 1.2826 - acc: 0.4182 - val_loss: 1.3935 - val_acc: 0.2419

Epoch 00

550/550 [==============================] - 0s 802us/step - loss: 0.8233 - acc: 0.6582 - val_loss: 3.2819 - val_acc: 0.1774

Epoch 00044: val_acc did not improve from 0.51613
Epoch 45/1000
550/550 [==============================] - 0s 801us/step - loss: 0.7974 - acc: 0.6582 - val_loss: 2.9016 - val_acc: 0.1774

Epoch 00045: val_acc did not improve from 0.51613
Epoch 46/1000
550/550 [==============================] - 0s 759us/step - loss: 0.7552 - acc: 0.7218 - val_loss: 3.0685 - val_acc: 0.1935

Epoch 00046: val_acc did not improve from 0.51613
Epoch 47/1000
550/550 [==============================] - 0s 740us/step - loss: 0.7526 - acc: 0.7236 - val_loss: 3.3362 - val_acc: 0.1452

Epoch 00047: val_acc did not improve from 0.51613
Epoch 48/1000
550/550 [==============================] - 0s 794us/step - loss: 0.7211 - acc: 0.7273 - val_loss: 3.0859 - val_acc: 0.1774

Epoch 00048: val_acc did not improve from 0.51613
Epoch 49/1000
550/550 [==============================] - 0s 802us/step - l


Epoch 00087: val_acc did not improve from 0.51613
Epoch 88/1000
550/550 [==============================] - 1s 1ms/step - loss: 0.3054 - acc: 0.8982 - val_loss: 5.9032 - val_acc: 0.1774

Epoch 00088: val_acc did not improve from 0.51613
Epoch 89/1000
550/550 [==============================] - 1s 1ms/step - loss: 0.3042 - acc: 0.8836 - val_loss: 5.3511 - val_acc: 0.1935

Epoch 00089: val_acc did not improve from 0.51613
Epoch 90/1000
550/550 [==============================] - 1s 1ms/step - loss: 0.2556 - acc: 0.9200 - val_loss: 5.8798 - val_acc: 0.1452

Epoch 00090: val_acc did not improve from 0.51613
Epoch 91/1000
550/550 [==============================] - 0s 868us/step - loss: 0.2681 - acc: 0.9164 - val_loss: 5.0471 - val_acc: 0.2097

Epoch 00091: val_acc did not improve from 0.51613
Epoch 92/1000
550/550 [==============================] - 0s 814us/step - loss: 0.2431 - acc: 0.9309 - val_loss: 5.4650 - val_acc: 0.2419

Epoch 00092: val_acc did not improve from 0.51613
Epoch 93/1000
5

550/550 [==============================] - 1s 973us/step - loss: 0.0937 - acc: 0.9800 - val_loss: 7.8019 - val_acc: 0.1935

Epoch 00131: val_acc did not improve from 0.51613
Epoch 132/1000
550/550 [==============================] - 0s 787us/step - loss: 0.0932 - acc: 0.9691 - val_loss: 7.6142 - val_acc: 0.1935

Epoch 00132: val_acc did not improve from 0.51613
Epoch 133/1000
550/550 [==============================] - 0s 807us/step - loss: 0.0945 - acc: 0.9764 - val_loss: 7.5075 - val_acc: 0.2097

Epoch 00133: val_acc did not improve from 0.51613
Epoch 134/1000
550/550 [==============================] - 0s 783us/step - loss: 0.0986 - acc: 0.9745 - val_loss: 7.8508 - val_acc: 0.1774

Epoch 00134: val_acc did not improve from 0.51613
Epoch 135/1000
550/550 [==============================] - 0s 795us/step - loss: 0.1118 - acc: 0.9709 - val_loss: 6.9574 - val_acc: 0.2258

Epoch 00135: val_acc did not improve from 0.51613
Epoch 136/1000
550/550 [==============================] - 0s 763us/ste

(612, 4)

In [14]:
y_train

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)